In [ ]:
import os, glob
import shutil, zipfile
import warnings
import pandas as pd
import datetime as dt
import pandasql
import numpy as np
from datetime import datetime
from pandasql import sqldf

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    #warnings.warn("DataFrame is highly fragmented", PerformanceWarning)

In [ ]:
bu = "BP"
cd ={"BP":"BPA907","BR":"BRA903","BD":"BDA902"}

In [ ]:
if bu not in cd:
    print("sair")

In [ ]:
# Buscando arquivos
origem = r'Y:\Reports'
destino = r'C:\Users\silmao\OneDrive - Dufry International AG\Documentos\BI'
os.chdir(destino)

In [ ]:
shutil.copy(origem+'\GESETPAR\GESETPAR-'+ bu +'.ZIP', destino)
shutil.copy(origem+'\INSTPAAR\INSTPAAR-'+ bu +'.ZIP', destino)

In [ ]:
with zipfile.ZipFile('GESETPAR-'+ bu +'.ZIP', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
shutil.move(zip_ref.namelist()[0], 'GESETPAR-'+ bu +'.XLS')

In [ ]:
with zipfile.ZipFile('INSTPAAR-'+ bu +'.ZIP', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
shutil.move(zip_ref.namelist()[0], 'INSTPAAR-'+ bu +'.XLS')

In [ ]:
def limpa(strLimpar):
    return str.strip(strLimpar)[2:-1]

In [ ]:
# GESETPAR
gesetpar = pd.read_table('GESETPAR-'+ bu +'.XLS', header = 0 , sep="\t", encoding='latin1', low_memory = False, usecols=[ 0, 1, 2, 5, 10, 15, 16, 17, 20, 22, 26,  31, 32, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 75, 76, 77, 78, 79, 80], converters={1:limpa, 79:limpa})

In [ ]:
gesetpar.head(10)

In [ ]:
gesetpar.columns = gesetpar.columns.map(str.strip)

In [ ]:
# Altera ordem colunas no dataframe
cols = [5, 6, 8, 0, 1, 2, 3, 4, 7, 9, 10, 11, 12, 25, 26, 27, 28, 29, 30, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24 ]
gesetpar = gesetpar[[gesetpar.columns[i] for i in cols]]

In [ ]:
gesetpar.columns

In [ ]:
gesetpar['GL.HI.CO.'] = gesetpar['GL.HI.CO.'].str[0:2]
gesetpar = gesetpar.rename(columns={"GL.HI.CO.":"CATEGORIA"})

In [ ]:
gesetpar['STATUS ART. IN BUSSINESS UNIT'] = gesetpar['STATUS ART. IN BUSSINESS UNIT'].map(str.strip)

In [ ]:
gesetpar['DESC STATUS'] = ['A' if i == '' else i for i in gesetpar['STATUS ART. IN BUSSINESS UNIT']]
status = str.maketrans({'*':'DESCONTINUADO','A':'ATIVO','N':'NOVO','L':'ATIVO','X':'ATIVO'})
gesetpar['DESC STATUS'] =  gesetpar['DESC STATUS'].str.translate(status)
gesetpar = gesetpar.rename(columns={"STATUS ART. IN BUSSINESS UNIT":"STATUS GAMMA"})
gesetpar.insert(8, "DESC STATUS", gesetpar.pop("DESC STATUS"))

In [ ]:
# INSTPAAR
instapaar = pd.read_table('INSTPAAR-'+ bu +'.XLS', sep="\t", encoding='latin1', low_memory = False, usecols=[0,1,5,9,10,16,17,18,19,20,32, 35], converters={5:limpa})

In [ ]:
instapaar.head(10)

In [ ]:
instapaar.columns = instapaar.columns.map(str.strip)

In [ ]:
centros = instapaar["Center"].unique()

In [ ]:
saldo_lojas = instapaar.loc[instapaar["Center"] != cd[bu], ("Article", "Curr Stock")]

In [ ]:
saldo_lojas["Curr Stock"] = saldo_lojas["Curr Stock"].map(str.strip)

In [ ]:
saldo_lojas["Curr Stock"] = saldo_lojas["Curr Stock"].str.replace(",00","")

In [ ]:
saldo_lojas["Curr Stock"] = saldo_lojas["Curr Stock"].astype(int)

In [ ]:
saldo_lojas = saldo_lojas.groupby(by="Article", as_index = False).sum()

In [ ]:
saldo_lojas.head(10)

In [ ]:
instapaar = instapaar.pivot(index="Article", columns=["Center","Center SAP"], values=["MAX","MIN","Curr Stock", "Transit"]).reset_index()

In [ ]:
instapaar = instapaar.set_axis(instapaar.columns.map('_'.join), axis=1, copy=False)

In [ ]:
instapaar = instapaar.rename(columns={"Article__": "Article"})

In [ ]:
instapaar.columns = instapaar.columns.map(str.strip)

In [ ]:
nova_ordem = [instapaar.columns[0]]
nova_ordem.append(instapaar.columns[1])
ret = [nova_ordem.append(c) for i in centros for c in instapaar.columns if i in c]

In [ ]:
instapaar = instapaar[nova_ordem]

In [ ]:
instapaar.columns[0:40]

In [ ]:
# Gerando df consolidado
df = gesetpar.merge(instapaar, left_on = "ART. CODE", right_on = "Article", how = "left")

In [ ]:
df = df.merge(saldo_lojas, on="Article", how="left")

In [ ]:
df = df.drop(["Article"], axis = 1)

In [ ]:
df.insert(13,"SALDO LOJAS", df.pop("Curr Stock"))
df["SALDO LOJAS"] = df["SALDO LOJAS"].fillna(0)

In [ ]:
df["SALDO LOJAS"] = df["SALDO LOJAS"].astype(int)

In [ ]:
col_cd = [i for i in df.columns if i.startswith ("Curr Stock_"+cd[bu])][0]

In [ ]:
# Remover coluna do estoque corrente no armazem e inserir em outra posição
df.insert(13,"SALDO CD", df.pop(col_cd))
df["SALDO CD"] = df["SALDO CD"].fillna(0)

In [ ]:
df = df.drop([t for t in df.columns if cd[bu] in t], axis = 1)

In [ ]:
df.insert(19,"MERCHANDISE TYPE", df.pop("MERCHANDISE TYPE"))

In [ ]:
df.head(10)

In [ ]:
df.columns[0:40]

In [ ]:
# Grava resultado final
saida = 'SUPPLY_CHAIN-'+bu+'.csv'

In [ ]:
df.to_csv(saida, sep=";", encoding="utf-8")

In [ ]:
saida_zip = "supply_chain_"+bu+".zip"
with zipfile.ZipFile(saida_zip, "w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip:
    zip.write(saida)

In [ ]:
shutil.copy(destino+'\\'+saida_zip, origem+'\\INSTPAAR\\')

In [ ]:
#FIM

In [ ]:
# referencias: se dataframe multiindex
df["SALDO CD"] = df[[t for t in [c for c in df.columns if isinstance(c, tuple)] if t[0]=='Curr Stock' and t[1] =="BPA907"][0]]

In [ ]:
# Remover coluna do estoque corrente no armazem e inserir na posição 13
df.insert(13, "SALDO CD", df.pop([t for t in [c for c in df.columns if isinstance(c, tuple)] if t[0]=='Curr Stock' and t[1] =="BPA907"][0]))

In [ ]:
# Remove demais colunas do BPA907
df = df.drop([t for t in [c for c in df.columns if isinstance(c, tuple)] if t[1] =="BPA907" or t[0] == "Article"], axis=1)

In [ ]:
# colocando uma coluna em outra posição
gesetpar.insert(0,"Categoria", gesetpar.pop("GL.HI.CO."))

In [ ]:
df.loc[df["SALDO LOJAS"] != 0, "SALDO LOJAS"]